In [3]:
# =====================================================
# 0. Imports & config
# =====================================================

# !pip install gender-guesser scikit-learn pandas numpy matplotlib

import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

import gender_guesser.detector as gender

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
plt.rcParams["figure.dpi"] = 120

# Output directory
output_dir = "./data/ml_pipeline_outputs"
os.makedirs(output_dir, exist_ok=True)

In [4]:
# =====================================================
# 1. Load dataset
# =====================================================

DATA_PATH = "./data/resume_data.csv"   # change if needed

df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
display(df.head())

# Label column must exist
LABEL_COL = "shortlist"
df[LABEL_COL] = df[LABEL_COL].astype(int)

Shape: (9544, 38)
Columns: ['name', 'address', 'career_objective', 'skills', 'educational_institution_name', 'degree_names', 'passing_years', 'educational_results', 'result_types', 'major_field_of_studies', 'professional_company_names', 'company_urls', 'start_dates', 'end_dates', 'years_of_experience', 'related_skils_in_job', 'positions', 'locations', 'responsibilities', 'extra_curricular_activity_types', 'extra_curricular_organization_names', 'extra_curricular_organization_links', 'role_positions', 'languages', 'proficiency_levels', 'certification_providers', 'certification_skills', 'online_links', 'issue_dates', 'expiry_dates', 'job_position_name', 'educationaL_requirements', 'experiencere_requirement', 'age_requirement', 'responsibilities.1', 'skills_required', 'matched_score', 'shortlist']


,name,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,...,issue_dates,expiry_dates,job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score,shortlist
0,Greta,NaN,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],...,NaN,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000,0
1,Kamau,NaN,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","['Mathematics', 'Statistics']",...,NaN,NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,At least 5 year(s),NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000,1
2,Patricia,NaN,NaN,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],['Electronics/Telecommunication'],...,NaN,NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),At least 3 years,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667,0
3,Elena,NaN,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],['Computer Applications'],...,NaN,NaN,Business Development Executive,Bachelor/Honors,1 to 3 years,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000,1
4,Zara,NaN,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],['Accounting'],...,[None],"['February 15, 2021']",Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000,1


In [5]:
# =====================================================
# 2. Name column & gender inference
# =====================================================

NAME_COL = "name"

detector = gender.Detector(case_sensitive=False)

def infer_gender(name: str) -> str:
    if pd.isna(name) or not isinstance(name, str) or name.strip() == "":
        return "unknown"
    first_name = name.strip().split()[0]
    g = detector.get_gender(first_name)
    if g in ["male", "mostly_male"]:
        return "male"
    elif g in ["female", "mostly_female"]:
        return "female"
    else:
        return "unknown"

df["gender_group"] = df[NAME_COL].apply(infer_gender)
print("\nInferred gender_group counts (before filtering):")
print(df["gender_group"].value_counts(dropna=False))

# 🔴 Keep only male / female, drop unknowns
df = df[df["gender_group"].isin(["male", "female"])].copy()
df.reset_index(drop=True, inplace=True)

print("\nAfter filtering to only male/female rows:")
print(df["gender_group"].value_counts(dropna=False))
print("New shape:", df.shape)


Inferred gender_group counts (before filtering):
gender_group
female     3589
male       3584
unknown    2371
Name: count, dtype: int64

After filtering to only male/female rows:
gender_group
female    3589
male      3584
Name: count, dtype: int64
New shape: (7173, 39)


In [6]:
# =====================================================
# 3. Build combined resume text
# =====================================================

# You can adjust which columns to include
text_cols = [
    "skills",
    "responsibilities.1",
    "educational_institution_name",
    "degree_names",
    "major_field_of_studies",
    "educational_results",
    "result_types",
]

text_cols = [c for c in text_cols if c in df.columns]

def combine_text_fields(row, cols):
    parts = []
    for c in cols:
        val = row.get(c, "")
        if pd.isna(val):
            val = ""
        parts.append(str(val))
    return " ".join(parts)

df["combined_text"] = df.apply(combine_text_fields, axis=1, cols=text_cols)
df["combined_text"] = df["combined_text"].fillna("").astype(str)

print("\nSample combined_text:")
display(df[["combined_text", LABEL_COL, "job_position_name"]].head())


Sample combined_text:


,combined_text,shortlist,job_position_name
0,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",0,Senior Software Engineer
1,"['Software Development', 'Machine Learning', '...",0,"Executive/ Senior Executive- Trade Marketing, ..."
2,"['accounts payables', 'accounts receivables', ...",1,Business Development Executive
3,"['Analytical reasoning', 'Compliance testing k...",1,Senior iOS Engineer
4,"['Machine Learning', 'Linear Regression', 'Rid...",0,Senior iOS Engineer


In [7]:
# =====================================================
# 4. Train / test split
# =====================================================

X = df["combined_text"]
y = df[LABEL_COL]

X_train_text, X_test_text, y_train, y_test, train_gender, test_gender = train_test_split(
    X,
    y,
    df["gender_group"],
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y,
)

print("\nTrain size:", len(X_train_text), "Test size:", len(X_test_text))


Train size: 5738 Test size: 1435


In [8]:
# =====================================================
# 5. TF-IDF vectorization
# =====================================================

vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words="english",
)

X_train_tfidf = vectorizer.fit_transform(X_train_text)
X_test_tfidf  = vectorizer.transform(X_test_text)

feature_names = np.array(vectorizer.get_feature_names_out())
print("TF-IDF shape (train):", X_train_tfidf.shape)
print("TF-IDF shape (test) :", X_test_tfidf.shape)

TF-IDF shape (train): (5738, 5000)
TF-IDF shape (test) : (1435, 5000)


In [9]:
# =====================================================
# 6. Helper: evaluation
# =====================================================

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

def evaluate_model(name, y_true, y_pred, verbose=True):
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0.0   # focus metric from proposal

    if verbose:
        print(f"\n===== {name} =====")
        print("Accuracy :", acc)
        print("Precision:", prec)
        print("Recall   :", rec)
        print("F1-score :", f1)
        print("FNR      :", fnr)
        print("\nClassification report:")
        print(classification_report(y_true, y_pred, zero_division=0))
        print("Confusion matrix (labels [0,1]):")
        print(cm)

    return {
        "model": name,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "FNR": fnr,
        "TP": tp,
        "FP": fp,
        "FN": fn,
        "TN": tn,
    }

In [10]:
# =====================================================
# 7. Train models
# =====================================================

metrics_summary = []

# 7.1 Linear SVM
linear_svm = LinearSVC(random_state=RANDOM_STATE, C=1.0)
linear_svm.fit(X_train_tfidf, y_train)
y_pred_linear = linear_svm.predict(X_test_tfidf)
metrics_summary.append(
    evaluate_model("Linear SVM (C=1.0)", y_test, y_pred_linear, verbose=True)
)

# 7.2 RBF SVM
rbf_svm = SVC(
    kernel="rbf",
    C=1.0,
    gamma="scale",
    probability=True,
    random_state=RANDOM_STATE,
)
rbf_svm.fit(X_train_tfidf, y_train)
y_pred_rbf = rbf_svm.predict(X_test_tfidf)
metrics_summary.append(
    evaluate_model("RBF SVM (C=1.0)", y_test, y_pred_rbf, verbose=True)
)

# 7.3 Gaussian NB
X_train_dense = X_train_tfidf.toarray()
X_test_dense  = X_test_tfidf.toarray()
gnb = GaussianNB()
gnb.fit(X_train_dense, y_train)
y_pred_gnb = gnb.predict(X_test_dense)
metrics_summary.append(
    evaluate_model("Gaussian NB", y_test, y_pred_gnb, verbose=True)
)

# Cross-validation for Linear SVM
print("\n===== Cross-validation: Linear SVM on training set =====")
cv_scores = cross_val_score(
    LinearSVC(random_state=RANDOM_STATE, C=1.0),
    X_train_tfidf,
    y_train,
    cv=5,
    scoring="f1",
)
print("CV F1 scores:", cv_scores)
print("Mean F1:", cv_scores.mean(), "Std:", cv_scores.std())

# Neat comparison table
metrics_df = pd.DataFrame(metrics_summary).set_index("model").round(3)
print("\n=== Benchmark comparison (test set) ===")
display(metrics_df)


===== Linear SVM (C=1.0) =====
Accuracy : 0.6306620209059234
Precision: 0.63671875
Recall   : 0.6608108108108108
F1-score : 0.6485411140583555
FNR      : 0.33918918918918917

Classification report:
              precision    recall  f1-score   support

           0       0.62      0.60      0.61       695
           1       0.64      0.66      0.65       740

    accuracy                           0.63      1435
   macro avg       0.63      0.63      0.63      1435
weighted avg       0.63      0.63      0.63      1435

Confusion matrix (labels [0,1]):
[[416 279]
 [251 489]]

===== RBF SVM (C=1.0) =====
Accuracy : 0.6425087108013937
Precision: 0.6396063960639606
Recall   : 0.7027027027027027
F1-score : 0.6696716033483581
FNR      : 0.2972972972972973

Classification report:
              precision    recall  f1-score   support

           0       0.65      0.58      0.61       695
           1       0.64      0.70      0.67       740

    accuracy                           0.64      14

,accuracy,precision,recall,f1,FNR,TP,FP,FN,TN
model,,,,,,,,,
Linear SVM (C=1.0),0.631,0.637,0.661,0.649,0.339,489,279,251,416
RBF SVM (C=1.0),0.643,0.640,0.703,0.670,0.297,520,293,220,402
Gaussian NB,0.587,0.633,0.477,0.544,0.523,353,205,387,490


In [11]:
# =====================================================
# 8. Interpretability: Linear SVM coefficients
# =====================================================

coefs = linear_svm.coef_[0]

def show_top_features(coefs, feature_names, top_k=20):
    top_pos_idx = np.argsort(coefs)[-top_k:][::-1]
    top_neg_idx = np.argsort(coefs)[:top_k]

    print("\nTop positive features (towards SHORTLIST = 1):")
    for idx in top_pos_idx:
        print(f"{feature_names[idx]:<25} {coefs[idx]:.4f}")

    print("\nTop negative features (towards SHORTLIST = 0):")
    for idx in top_neg_idx:
        print(f"{feature_names[idx]:<25} {coefs[idx]:.4f}")

show_top_features(coefs, feature_names, top_k=20)

# Example explanation function
def explain_prediction(text, model, vectorizer, feature_names, coefs, top_k=10):
    vec = vectorizer.transform([text])
    decision = model.decision_function(vec)[0]
    pred_label = model.predict(vec)[0]

    vec_dense = vec.toarray()[0]
    contributions = coefs * vec_dense

    non_zero_indices = np.where(vec_dense != 0)[0]
    non_zero_contribs = contributions[non_zero_indices]
    non_zero_features = feature_names[non_zero_indices]

    sorted_idx = np.argsort(np.abs(non_zero_contribs))[::-1][:top_k]

    print("\n=== Explanation for prediction ===")
    print("Predicted label   :", pred_label)
    print("Decision function :", decision)
    print("\nTop contributing features:")
    for i in sorted_idx:
        fname = non_zero_features[i]
        contrib = non_zero_contribs[i]
        print(f"{fname:<25} contribution: {contrib:.4f}")

example_text = X_test_text.iloc[0]
print("\nExample resume snippet:")
print(example_text[:400], "...\n")
explain_prediction(
    example_text,
    model=linear_svm,
    vectorizer=vectorizer,
    feature_names=feature_names,
    coefs=coefs,
    top_k=10,
)


Top positive features (towards SHORTLIST = 1):
preparation university    1.4078
sql machine               1.0732
knowledge university      1.0661
management machine        0.8302
maintenance university    0.8127
seo university            0.7921
metallurgy                0.7529
tech metallurgy           0.7529
analytics machine         0.7245
sql html                  0.6411
skills project            0.6399
html                      0.6338
intelligence data         0.6332
program                   0.6168
management                0.6095
degree                    0.5821
gpa                       0.5797
leadership                0.5680
analysis design           0.5593
skill                     0.5559

Top negative features (towards SHORTLIST = 0):
analysis university       -1.0960
learning university       -0.8646
institute engineering     -0.8185
verification university   -0.7813
quality university        -0.7632
integration university    -0.7625
learning software         -0.7260
analyt

In [12]:
# =====================================================
# 9. Fairness by gender_group (Linear SVM)
# =====================================================

def group_metrics(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0.0
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    return {
        "FPR": fpr,
        "FNR": fnr,
        "Precision": prec,
        "Recall": rec,
        "F1": f1,
        "Support": len(y_true),
    }

test_df = pd.DataFrame({
    "text": X_test_text.values,
    "y_true": y_test.values,
    "y_pred": y_pred_linear,
    "gender_group": test_gender.values,
})

fairness_results = []
for g in sorted(test_df["gender_group"].unique()):
    sub = test_df[test_df["gender_group"] == g]
    m = group_metrics(sub["y_true"], sub["y_pred"])
    m["gender_group"] = g
    fairness_results.append(m)

fairness_df = pd.DataFrame(fairness_results)
print("\n=== Fairness audit by gender_group (Linear SVM) ===")
display(fairness_df.set_index("gender_group"))


=== Fairness audit by gender_group (Linear SVM) ===


,FPR,FNR,Precision,Recall,F1,Support
gender_group,,,,,,
female,0.406685,0.313984,0.640394,0.686016,0.662420,738
male,0.395833,0.365651,0.632597,0.634349,0.633472,697


In [13]:
# =====================================================
# 10. Robustness attacks (Linear SVM)
# =====================================================

def attack_keyword_stuffing(text, keywords, repeat=3):
    stuffing = " ".join(list(keywords) * repeat)
    return text + " " + stuffing

def attack_random_typos(text, prob=0.02):
    chars = list(text)
    for i in range(len(chars)):
        if np.random.rand() < prob and chars[i].isalpha():
            chars.insert(i, chars[i])
    return "".join(chars)

def attack_template_header_footer(text):
    header = "Highly motivated candidate seeking challenging role. "
    footer = " Proven track record of excellence across multiple domains."
    return header + text + footer

def attack_synonym_replacement(text):
    mapping = {
        "good": "excellent",
        "great": "outstanding",
        "hardworking": "diligent",
        "team": "group",
        "leader": "lead",
    }
    words = text.split()
    new_words = [mapping.get(w.lower(), w) for w in words]
    return " ".join(new_words)

def attack_sentence_shuffle(text):
    sentences = [s.strip() for s in text.split(".") if s.strip() != ""]
    if len(sentences) <= 1:
        return text
    np.random.shuffle(sentences)
    return ". ".join(sentences) + "."

def attack_adversarial_insert(text):
    adv = (
        " consistently rated top performer with strong problem-solving, "
        "stakeholder management, and leadership skills "
    )
    mid = len(text) // 2
    return text[:mid] + adv + text[mid:]

def evaluate_attack(model, vectorizer, texts, labels, attack_fn, attack_name, **kwargs):
    X_base = vectorizer.transform(texts)
    base_pred = model.predict(X_base)

    rejected_mask = (base_pred == 0)
    rejected_texts = np.array(texts)[rejected_mask]

    if len(rejected_texts) == 0:
        return {
            "attack": attack_name,
            "flip_rate": 0.0,
            "flips": 0,
            "total": 0,
        }

    adv_texts = [attack_fn(t, **kwargs) for t in rejected_texts]
    X_adv = vectorizer.transform(adv_texts)
    adv_pred = model.predict(X_adv)

    flips = np.sum(adv_pred == 1)
    flip_rate = flips / len(adv_pred)

    return {
        "attack": attack_name,
        "flip_rate": flip_rate,
        "flips": flips,
        "total": len(adv_pred),
    }

np.random.seed(42)
top_keywords_for_attack = feature_names[np.argsort(coefs)[-10:][::-1]]

attacks = [
    ("Keyword stuffing",      attack_keyword_stuffing,      {"keywords": top_keywords_for_attack, "repeat": 3}),
    ("Random typos",          attack_random_typos,          {"prob": 0.02}),
    ("Template header/footer",attack_template_header_footer,{}),
    ("Synonym replacement",   attack_synonym_replacement,   {}),
    ("Sentence shuffle",      attack_sentence_shuffle,      {}),
    ("Adversarial insert",    attack_adversarial_insert,    {}),
]

attack_results = []
for name, fn, kw in attacks:
    res = evaluate_attack(
        model=linear_svm,
        vectorizer=vectorizer,
        texts=X_test_text.tolist(),
        labels=y_test.tolist(),
        attack_fn=fn,
        attack_name=name,
        **kw,
    )
    attack_results.append(res)

attack_results_df = pd.DataFrame(attack_results)
print("\nRobustness results (raw):")
display(attack_results_df)

# Nicely formatted robustness table
robustness_table = attack_results_df.rename(columns={
    "attack": "attack",
    "flip_rate": "flip_rate",
    "flips": "flips",
    "total": "total",
})

# Save robustness CSV
robustness_csv_path = os.path.join(output_dir, "robustness_report.csv")
robustness_table.to_csv(robustness_csv_path, index=False)
print(f"\nSaved robustness report CSV: {robustness_csv_path}")

Cs = [0.01, 0.1, 1.0, 10.0, 100.0]

robustness_vs_C = []

print("\n===== Robustness vs C (Linear SVM, keyword stuffing attack) =====")

for C_val in Cs:
    # Train Linear SVM with given C
    svm_C = LinearSVC(random_state=RANDOM_STATE, C=C_val)
    svm_C.fit(X_train_tfidf, y_train)

    # Test-set performance
    X_test_tfidf = vectorizer.transform(X_test_text)
    y_pred_C = svm_C.predict(X_test_tfidf)
    metrics_C = evaluate_model(
        name=f"Linear SVM (C={C_val})",
        y_true=y_test,
        y_pred=y_pred_C,
        verbose=False,   # already printed once for C=1.0
    )

    # For a fair comparison, reuse the same keyword list we used before
    # (top_keywords_for_attack defined earlier from the baseline model)
    kw_attack_res = evaluate_attack(
        model=svm_C,
        vectorizer=vectorizer,
        texts=X_test_text.tolist(),
        labels=y_test.tolist(),
        attack_fn=attack_keyword_stuffing,
        attack_name="Keyword stuffing",
        keywords=top_keywords_for_attack,
        repeat=3,
    )

    robustness_vs_C.append({
        "C": C_val,
        "accuracy": metrics_C["accuracy"],
        "FNR": metrics_C["FNR"],
        "flip_rate_keyword_stuffing": kw_attack_res["flip_rate"],
    })

robustness_vs_C_df = pd.DataFrame(robustness_vs_C)
print("\nRobustness vs C (table):")
display(robustness_vs_C_df)

# Save CSV
robustness_vs_C_csv = os.path.join(output_dir, "robustness_vs_C.csv")
robustness_vs_C_df.to_csv(robustness_vs_C_csv, index=False)
print(f"Saved robustness-vs-C report CSV: {robustness_vs_C_csv}")

# Plot: C vs accuracy / FNR / flip rate
fig, ax1 = plt.subplots(figsize=(7, 4))

ax1.set_xscale("log")
ax1.plot(
    robustness_vs_C_df["C"],
    robustness_vs_C_df["accuracy"],
    marker="o",
    label="Accuracy"
)
ax1.plot(
    robustness_vs_C_df["C"],
    robustness_vs_C_df["FNR"],
    marker="o",
    label="FNR"
)
ax1.set_xlabel("C (log scale)")
ax1.set_ylabel("Accuracy / FNR")
ax1.set_title("Effect of C on performance and robustness (Linear SVM)")

ax2 = ax1.twinx()
ax2.plot(
    robustness_vs_C_df["C"],
    robustness_vs_C_df["flip_rate_keyword_stuffing"],
    marker="s",
    linestyle="--",
    label="Flip rate (keyword stuffing)"
)
ax2.set_ylabel("Flip rate (keyword stuffing)")

# Build a combined legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="best")

plt.tight_layout()
robustness_vs_C_plot = os.path.join(output_dir, "robustness_vs_C.png")
fig.savefig(robustness_vs_C_plot, bbox_inches="tight")
plt.close(fig)

print(f"Saved robustness-vs-C plot: {robustness_vs_C_plot}")


Robustness results (raw):


,attack,flip_rate,flips,total
0,Keyword stuffing,1.000000,667,667
1,Random typos,0.082459,55,667
2,Template header/footer,0.188906,126,667
3,Synonym replacement,0.001499,1,667
4,Sentence shuffle,0.005997,4,667
5,Adversarial insert,0.337331,225,667



Saved robustness report CSV: ./data/ml_pipeline_outputs/robustness_report.csv

===== Robustness vs C (Linear SVM, keyword stuffing attack) =====

Robustness vs C (table):


,C,accuracy,FNR,flip_rate_keyword_stuffing
0,0.01,0.646690,0.263514,0.169550
1,0.10,0.639721,0.305405,0.987302
2,1.00,0.630662,0.339189,1.000000
3,10.00,0.629268,0.340541,1.000000
4,100.00,0.628571,0.339189,1.000000


Saved robustness-vs-C report CSV: ./data/ml_pipeline_outputs/robustness_vs_C.csv
Saved robustness-vs-C plot: ./data/ml_pipeline_outputs/robustness_vs_C.png


In [14]:
# =====================================================
# 11. Requirement-matching features & bias: good vs weak
# =====================================================

# Helpers already imported: re, np, pd

def parse_skill_list(s):
    if pd.isna(s):
        return set()
    tokens = re.split(r"[,\;/\|]", str(s).lower())
    return set(t.strip() for t in tokens if t.strip() != "")

def parse_years_from_text(s):
    if pd.isna(s):
        return np.nan
    nums = re.findall(r"\d+\.?\d*", str(s))
    if not nums:
        return np.nan
    return float(nums[0])

def degree_level(text):
    if pd.isna(text):
        return 0
    t = str(text).lower()
    if "phd" in t or "doctor" in t:
        return 4
    if "master" in t or "m.tech" in t or "mtech" in t or "m.sc" in t:
        return 3
    if "bachelor" in t or "b.tech" in t or "btech" in t or "b.e" in t or "bsc" in t:
        return 2
    if "diploma" in t:
        return 1
    return 0

def parse_age_requirement(text):
    if pd.isna(text):
        return (np.nan, np.nan)
    t = str(text).lower()
    nums = re.findall(r"\d+", t)
    if ("between" in t or "-" in t) and len(nums) >= 2:
        return (float(nums[0]), float(nums[1]))
    if "below" in t or "under" in t or "upto" in t:
        return (np.nan, float(nums[0])) if nums else (np.nan, np.nan)
    if "above" in t or "over" in t:
        return (float(nums[0]), np.nan) if nums else (np.nan, np.nan)
    if len(nums) == 1:
        return (float(nums[0]), np.nan)
    return (np.nan, np.nan)

# Skills overlap
cand_skills = df["skills"].fillna("")
req_skills  = df["skills_required"].fillna("")

skill_overlap_count = []
skill_overlap_ratio_req = []
missing_required_skill_count = []
skills_meet = []

for cs, rs in zip(cand_skills, req_skills):
    cs_set = parse_skill_list(cs)
    rs_set = parse_skill_list(rs)
    inter = cs_set & rs_set

    overlap = len(inter)
    missing = max(len(rs_set) - overlap, 0)
    ratio_req = overlap / len(rs_set) if len(rs_set) > 0 else 0.0
    meet_flag = 1 if missing == 0 and len(rs_set) > 0 else 0

    skill_overlap_count.append(overlap)
    skill_overlap_ratio_req.append(ratio_req)
    missing_required_skill_count.append(missing)
    skills_meet.append(meet_flag)

df["skill_overlap_count"] = skill_overlap_count
df["skill_overlap_ratio_req"] = skill_overlap_ratio_req
df["missing_required_skill_count"] = missing_required_skill_count
df["skills_meet_all_required"] = skills_meet

# Experience
cand_exp = pd.to_numeric(df["years_of_experience"], errors="coerce")
req_exp_raw = df["experiencere_requirement"].fillna("")
req_exp = req_exp_raw.apply(parse_years_from_text)

df["candidate_experience_years"] = cand_exp
df["required_experience_years"] = req_exp
df["experience_gap"] = df["candidate_experience_years"] - df["required_experience_years"]
df["experience_meets"] = (df["experience_gap"] >= 0).astype(int)

# Education
cand_edu_text = (
    df["educational_institution_name"].fillna("").astype(str)
    + " " + df["degree_names"].fillna("").astype(str)
    + " " + df["major_field_of_studies"].fillna("").astype(str)
    + " " + df["educational_results"].fillna("").astype(str)
    + " " + df["result_types"].fillna("").astype(str)
)

req_edu_text = df["educationaL_requirements"].fillna("")

df["candidate_education_level"] = cand_edu_text.apply(degree_level)
df["required_education_level"] = req_edu_text.apply(degree_level)
df["education_gap"] = df["candidate_education_level"] - df["required_education_level"]
df["education_meets"] = (df["education_gap"] >= 0).astype(int)

# Age requirement (job-level only)
age_req_text = df["age_requirement"].fillna("")
age_mins = []
age_maxs = []
for t in age_req_text:
    mn, mx = parse_age_requirement(t)
    age_mins.append(mn)
    age_maxs.append(mx)

df["age_min_required"] = age_mins
df["age_max_required"] = age_maxs
df["has_age_requirement"] = (~df["age_requirement"].isna()).astype(int)

# Responsibilities similarity (simple Jaccard between skills & responsibilities)
def jaccard_tokens(a, b):
    if pd.isna(a) or pd.isna(b):
        return 0.0
    a_set = set(str(a).lower().split())
    b_set = set(str(b).lower().split())
    if not a_set or not b_set:
        return 0.0
    inter = len(a_set & b_set)
    union = len(a_set | b_set)
    return inter / union

df["responsibility_match_jaccard"] = [
    jaccard_tokens(cand, job_req)
    for cand, job_req in zip(
        df["skills"].fillna(""), df["responsibilities.1"].fillna("")
    )
]

df["matched_score"] = pd.to_numeric(df["matched_score"], errors="coerce")

# Good vs weak
ms_valid = df["matched_score"].dropna()
if not ms_valid.empty:
    ms_threshold = ms_valid.quantile(0.7)
else:
    ms_threshold = None

df["high_matched_score"] = (
    (df["matched_score"] >= ms_threshold).astype(int)
    if ms_threshold is not None
    else 0
)

df["meets_all_requirements"] = (
    (df["skills_meet_all_required"] == 1)
    & (df["experience_meets"] == 1)
    & (df["education_meets"] == 1)
).astype(int)

df["good_candidate"] = (
    (df["meets_all_requirements"] == 1) | (df["high_matched_score"] == 1)
).astype(int)
df["weak_candidate"] = 1 - df["good_candidate"]

df["is_shortlisted"] = df[LABEL_COL].astype(int)
df["good_but_rejected"] = (
    (df["good_candidate"] == 1) & (df["is_shortlisted"] == 0)
).astype(int)
df["weak_but_shortlisted"] = (
    (df["good_candidate"] == 0) & (df["is_shortlisted"] == 1)
).astype(int)

print("\n=== Requirement vs Shortlisting summary (overall) ===")
total_good = df["good_candidate"].sum()
total_weak = df["weak_candidate"].sum()
print(f"Total good candidates (by requirements/match): {total_good}")
print(f"Total weak candidates                         : {total_weak}")
print(f"Good but rejected                             : {df['good_but_rejected'].sum()}")
print(f"Weak but shortlisted                          : {df['weak_but_shortlisted'].sum()}")

if "gender_group" in df.columns:
    summary_by_gender = df.groupby("gender_group").agg(
        count=("is_shortlisted", "size"),
        good_candidates=("good_candidate", "sum"),
        weak_candidates=("weak_candidate", "sum"),
        good_but_rejected=("good_but_rejected", "sum"),
        weak_but_shortlisted=("weak_but_shortlisted", "sum"),
        avg_matched_score=("matched_score", "mean"),
        avg_skill_overlap=("skill_overlap_ratio_req", "mean"),
    )
    print("\n=== Requirement vs Shortlisting by gender_group ===")
    print(summary_by_gender)


=== Requirement vs Shortlisting summary (overall) ===
Total good candidates (by requirements/match): 2251
Total weak candidates                         : 4922
Good but rejected                             : 352
Weak but shortlisted                          : 1802

=== Requirement vs Shortlisting by gender_group ===
              count  good_candidates  weak_candidates  good_but_rejected  \
gender_group                                                               
female         3589             1129             2460                179   
male           3584             1122             2462                173   

              weak_but_shortlisted  avg_matched_score  avg_skill_overlap  
gender_group                                                              
female                         890           0.660678                0.0  
male                           912           0.660177                0.0  


In [15]:
# =====================================================
# 12. Save some plots (dashboard + robustness + features + CM)
# =====================================================

from sklearn.metrics import ConfusionMatrixDisplay

all_models_for_cm = [
    ("Linear SVM (C=1.0)", y_pred_linear),
    ("RBF SVM (C=1.0)",    y_pred_rbf),
    ("Gaussian NB",        y_pred_gnb),
]

for name, y_pred in all_models_for_cm:
    cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
    disp = ConfusionMatrixDisplay(cm, display_labels=[0, 1])
    fig, ax = plt.subplots(figsize=(4.5, 4))
    disp.plot(ax=ax, cmap="viridis", colorbar=True)
    ax.set_title(f"Confusion Matrix ({name})")
    plt.tight_layout()
    cm_file = os.path.join(
        output_dir,
        f"confusion_matrix_{name.split()[0].lower().replace('(', '').replace(')', '')}.png"
    )
    fig.savefig(cm_file, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved confusion matrix for {name}: {cm_file}")

# Top 15 features for plot
top_k = 15
top_idx = np.argsort(np.abs(coefs))[-top_k:]
top_features = feature_names[top_idx]
top_values = coefs[top_idx]
order = np.argsort(top_values)
top_features = top_features[order]
top_values = top_values[order]

cm_svc = confusion_matrix(y_test, y_pred_rbf, labels=[0, 1])

# Fairness plot data
fairness_plot = fairness_df.set_index("gender_group")[["FPR", "FNR"]]
groups = fairness_plot.index.tolist()
x = np.arange(len(groups))
width = 0.35

# Dashboard
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# (1,1) Fairness
axes[0, 0].bar(x - width/2, fairness_plot["FPR"].values, width, label="FPR")
axes[0, 0].bar(x + width/2, fairness_plot["FNR"].values, width, label="FNR")
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(groups)
axes[0, 0].set_ylabel("Rate")
axes[0, 0].set_title("Fairness by group (FPR & FNR)")
axes[0, 0].legend()

# (1,2) Robustness
axes[0, 1].bar(robustness_table["attack"], robustness_table["flip_rate"])
for i, v in enumerate(robustness_table["flip_rate"].values):
    axes[0, 1].text(i, v + 0.0005, f"{v:.3f}", ha="center", va="bottom", fontsize=8)
axes[0, 1].set_ylabel("Flip rate")
axes[0, 1].set_title("Robustness: Flip rates by attack")
axes[0, 1].set_xticklabels(robustness_table["attack"], rotation=25, ha="right")

# (2,1) Top features
axes[1, 0].barh(top_features, top_values)
axes[1, 0].set_title("Top 15 model features (coef magnitude)")
axes[1, 0].set_xlabel("Coefficient")
axes[1, 0].invert_yaxis()

# (2,2) Confusion matrix
im = axes[1, 1].imshow(cm_svc, interpolation="nearest")
axes[1, 1].set_title("Confusion Matrix (RBF SVM)")
axes[1, 1].set_xlabel("Predicted")
axes[1, 1].set_ylabel("Actual")
axes[1, 1].set_xticks([0, 1])
axes[1, 1].set_yticks([0, 1])
axes[1, 1].set_xticklabels([0, 1])
axes[1, 1].set_yticklabels([0, 1])
for i in range(cm_svc.shape[0]):
    for j in range(cm_svc.shape[1]):
        axes[1, 1].text(j, i, str(cm_svc[i, j]), ha="center", va="center", color="black")
fig.colorbar(im, ax=axes[1, 1])

plt.tight_layout()
dashboard_path = os.path.join(output_dir, "dashboard_summary.png")
fig.savefig(dashboard_path, bbox_inches="tight")
plt.close(fig)

# Separate robustness plot
fig, ax = plt.subplots(figsize=(7, 4))
ax.bar(robustness_table["attack"], robustness_table["flip_rate"])
for i, v in enumerate(robustness_table["flip_rate"].values):
    ax.text(i, v + 0.0005, f"{v:.3f}", ha="center", va="bottom", fontsize=8)
ax.set_ylabel("Flip rate")
ax.set_title("Robustness: Flip rates by attack")
ax.set_xticklabels(robustness_table["attack"], rotation=25, ha="right")
plt.tight_layout()
robustness_plot_path = os.path.join(output_dir, "robustness_flip_rates_pretty.png")
fig.savefig(robustness_plot_path, bbox_inches="tight")
plt.close(fig)

# Separate top features plot
fig, ax = plt.subplots(figsize=(7, 5))
ax.barh(top_features, top_values)
ax.set_title("Top 15 model features (coef magnitude)")
ax.set_xlabel("Coefficient")
ax.invert_yaxis()
plt.tight_layout()
top_features_path = os.path.join(output_dir, "top_features.png")
fig.savefig(top_features_path, bbox_inches="tight")
plt.close(fig)

# Separate confusion matrix
fig, ax = plt.subplots(figsize=(4.5, 4))
im = ax.imshow(cm_svc, interpolation="nearest")
ax.set_title("Confusion Matrix (RBF SVM)")
ax.set_xlabel("Predicted")
ax.set_ylabel("Actual")
ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels([0, 1])
ax.set_yticklabels([0, 1])
for i in range(cm_svc.shape[0]):
    for j in range(cm_svc.shape[1]):
        ax.text(j, i, str(cm_svc[i, j]), ha="center", va="center", color="black")
fig.colorbar(im)
plt.tight_layout()
cm_path = os.path.join(output_dir, "confusion_matrix_small.png")
fig.savefig(cm_path, bbox_inches="tight")
plt.close(fig)

print(f"\nSaved dashboard: {dashboard_path}")
print(f"Saved robustness plot: {robustness_plot_path}")
print(f"Saved top features plot: {top_features_path}")
print(f"Saved confusion matrix: {cm_path}")

print("\nRobustness results (table):")
print(robustness_table)

Saved confusion matrix for Linear SVM (C=1.0): ./data/ml_pipeline_outputs/confusion_matrix_linear.png
Saved confusion matrix for RBF SVM (C=1.0): ./data/ml_pipeline_outputs/confusion_matrix_rbf.png
Saved confusion matrix for Gaussian NB: ./data/ml_pipeline_outputs/confusion_matrix_gaussian.png


/var/folders/fx/jncymvk542l1dbmsb4v40p_c0000gn/T/ipykernel_74260/2975014338.py:63: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axes[0, 1].set_xticklabels(robustness_table["attack"], rotation=25, ha="right")
/var/folders/fx/jncymvk542l1dbmsb4v40p_c0000gn/T/ipykernel_74260/2975014338.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(robustness_table["attack"], rotation=25, ha="right")



Saved dashboard: ./data/ml_pipeline_outputs/dashboard_summary.png
Saved robustness plot: ./data/ml_pipeline_outputs/robustness_flip_rates_pretty.png
Saved top features plot: ./data/ml_pipeline_outputs/top_features.png
Saved confusion matrix: ./data/ml_pipeline_outputs/confusion_matrix_small.png

Robustness results (table):
                   attack  flip_rate  flips  total
0        Keyword stuffing   1.000000    667    667
1            Random typos   0.082459     55    667
2  Template header/footer   0.188906    126    667
3     Synonym replacement   0.001499      1    667
4        Sentence shuffle   0.005997      4    667
5      Adversarial insert   0.337331    225    667


In [16]:
# =====================================================
# 13. Human-readable global summary
# =====================================================

# Use metrics_df we created earlier
lin = metrics_df.loc["Linear SVM (C=1.0)"]
rbf = metrics_df.loc["RBF SVM (C=1.0)"]
nb  = metrics_df.loc["Gaussian NB"]

print("\n================= HUMAN-READABLE SUMMARY =================\n")

print("1) Overall performance of the three models on the held-out test set:")
print(f"   • Linear SVM (main, interpretable model):")
print(f"       - Accuracy: {lin['accuracy']*100:.1f}%")
print(f"       - Precision (shortlist=1): {lin['precision']*100:.1f}%")
print(f"       - Recall (shortlist=1):    {lin['recall']*100:.1f}%")
print(f"       - F1 score:                {lin['f1']:.3f}")
print(f"       - FNR (missed good ones):  {lin['FNR']*100:.1f}%\n")

print(f"   • RBF SVM (higher-capacity black-box benchmark):")
print(f"       - Accuracy: {rbf['accuracy']*100:.1f}%, F1: {rbf['f1']:.3f}, FNR: {rbf['FNR']*100:.1f}%")

print(f"   • Gaussian NB (simple probabilistic baseline):")
print(f"       - Accuracy: {nb['accuracy']*100:.1f}%, F1: {nb['f1']:.3f}, FNR: {nb['FNR']*100:.1f}%\n")

mean_cv = cv_scores.mean()
std_cv = cv_scores.std()
print("2) Stability across data splits (cross-validation on Linear SVM):")
print(f"   • Average F1 across folds: {mean_cv:.3f} (± {std_cv:.3f}).\n")


================= HUMAN-READABLE SUMMARY =================

1) Overall performance of the three models on the held-out test set:
   • Linear SVM (main, interpretable model):
       - Accuracy: 63.1%
       - Precision (shortlist=1): 63.7%
       - Recall (shortlist=1):    66.1%
       - F1 score:                0.649
       - FNR (missed good ones):  33.9%

   • RBF SVM (higher-capacity black-box benchmark):
       - Accuracy: 64.3%, F1: 0.670, FNR: 29.7%
   • Gaussian NB (simple probabilistic baseline):
       - Accuracy: 58.7%, F1: 0.544, FNR: 52.3%

2) Stability across data splits (cross-validation on Linear SVM):
   • Average F1 across folds: 0.650 (± 0.005).



In [17]:
# =====================================================
# 14. Single candidate explanation (requirements + model + bias)
# =====================================================

def explain_random_candidate(
    model,
    vectorizer,
    feature_names,
    coefs,
    df,
    X_text,
    y_true,
    gender_series,
    fairness_df,
    name_col=NAME_COL,
    top_local_words=8,
):
    rand_pos = np.random.randint(0, len(X_text))
    idx = X_text.index[rand_pos]

    text = X_text.loc[idx]
    true_label = int(y_true.loc[idx])
    gender_group = gender_series.loc[idx]
    row = df.loc[idx]

    vec = vectorizer.transform([text])
    decision = model.decision_function(vec)[0]
    pred_label = int(model.predict(vec)[0])

    vec_dense = vec.toarray()[0]
    contributions = coefs * vec_dense
    non_zero_idx = np.where(vec_dense != 0)[0]
    non_zero_contribs = contributions[non_zero_idx]
    non_zero_features = feature_names[non_zero_idx]

    pos_mask = non_zero_contribs > 0
    neg_mask = non_zero_contribs < 0
    pos_features = non_zero_features[pos_mask]
    pos_values = non_zero_contribs[pos_mask]
    neg_features = non_zero_features[neg_mask]
    neg_values = non_zero_contribs[neg_mask]

    pos_order = np.argsort(-pos_values)[:top_local_words]
    neg_order = np.argsort(neg_values)[:top_local_words]

    top_help = list(zip(pos_features[pos_order], pos_values[pos_order]))
    top_hurt = list(zip(neg_features[neg_order], neg_values[neg_order]))

    skills_meet = int(row.get("skills_meet_all_required", 0))
    exp_meet = int(row.get("experience_meets", 0))
    edu_meet = int(row.get("education_meets", 0))
    good_candidate = int(row.get("good_candidate", 0))
    weak_candidate = int(row.get("weak_candidate", 0))
    matched_score = row.get("matched_score", np.nan)

    exp_gap = row.get("experience_gap", np.nan)
    edu_gap = row.get("education_gap", np.nan)
    skill_overlap = row.get("skill_overlap_ratio_req", np.nan)
    missing_skills = int(row.get("missing_required_skill_count", 0))
    age_req = row.get("age_requirement", None)
    has_age_req = int(row.get("has_age_requirement", 0))

    fairness_row = None
    if gender_group in fairness_df["gender_group"].values:
        fairness_row = fairness_df[fairness_df["gender_group"] == gender_group].iloc[0]

    print("\n================= CANDIDATE EXPLANATION =================\n")

    name = row.get(name_col, "Unknown")
    jp = row.get("job_position_name", "Unknown position")

    print(f"Candidate: {name}  (index {idx})")
    print(f"Applied for: {jp}")
    print(f"Inferred group (from name): {gender_group}")
    print(f"True label in dataset      : {'SHORTLIST' if true_label == 1 else 'REJECT'}")
    print(f"Model prediction           : {'SHORTLIST' if pred_label == 1 else 'REJECT'}")
    print(f"Decision score (margin)    : {decision:.3f}\n")

    print("Requirement satisfaction:")
    print(f"  • Skills requirement met?       {'YES' if skills_meet else 'NO'}")
    print(f"  • Experience requirement met?   {'YES' if exp_meet else 'NO'}")
    print(f"  • Education requirement met?    {'YES' if edu_meet else 'NO'}")
    if not np.isnan(exp_gap):
        print(f"  • Experience gap (cand - req):  {exp_gap:.1f} years")
    if not np.isnan(edu_gap):
        print(f"  • Education gap (cand - req):   {edu_gap:.1f} level(s)")
    if not np.isnan(skill_overlap):
        print(f"  • Skill overlap (of required):  {skill_overlap*100:.1f}%")
        print(f"  • Missing required skills:      {missing_skills}")
    if has_age_req and isinstance(age_req, str):
        print(f"  • Job age requirement:          \"{age_req}\"")
    else:
        print("  • Job has no explicit age restriction or it's unspecified.")

    print("\nOverall requirement-based verdict:")
    if good_candidate:
        print("  → This candidate looks GOOD based on requirements / matched_score.")
    else:
        print("  → This candidate looks WEAK / borderline based on requirements / matched_score.")
    if not np.isnan(matched_score):
        print(f"  Matched score: {matched_score:.3f}")

    print("\nHow the model decided (from text):")
    if pred_label == 1:
        print("  The model chose to SHORTLIST this candidate.")
    else:
        print("  The model chose to REJECT this candidate.")

    print("\nTop words/phrases that helped:")
    if top_help:
        for w, v in top_help:
            print(f"   + '{w}' (towards shortlist, contribution {v:.4f})")
    else:
        print("   + No strong positive cues detected.")

    print("\nTop words/phrases that hurt:")
    if top_hurt:
        for w, v in top_hurt:
            print(f"   - '{w}' (towards reject, contribution {v:.4f})")
    else:
        print("   - No strong negative cues detected.")

    print("\nConsistency between requirements and decision:")
    if good_candidate and pred_label == 0:
        print("  → GOOD candidate but model REJECTED them.")
        print("     Possible reasons: resume text doesn't highlight strengths well,")
        print("     or the model/historical labels encode bias against similar profiles.")
    elif not good_candidate and pred_label == 1:
        print("  → WEAK candidate but model SHORTLISTED them.")
        print("     Possible reasons: buzzwords in text overpower actual requirement mismatch,")
        print("     or training data historically favoured similar weak profiles.")
    else:
        print("  → Model decision roughly aligns with requirement-based assessment.")

    print("\nDemographic context:")
    if fairness_row is not None:
        fpr = fairness_row["FPR"] * 100
        fnr = fairness_row["FNR"] * 100
        support = int(fairness_row["Support"])
        print(
            f"   For group '{gender_group}':\n"
            f"    • FPR (wrongly shortlisted) ≈ {fpr:.1f}%\n"
            f"    • FNR (wrongly rejected)    ≈ {fnr:.1f}%\n"
            f"    • Test-set size for group   : {support}"
        )
    else:
        print("   No fairness statistics available for this group.")

    print("\nPlain-language takeaway:")
    if good_candidate and pred_label == 0:
        print("   Strong by requirements, but still rejected → classic unfair-looking case.")
    elif not good_candidate and pred_label == 1:
        print("   Weak by requirements, but still shortlisted → model might be overfitting to buzzwords.")
    else:
        print("   Nothing dramatic: model and requirements more or less agree for this candidate.")

    print("\n================= END CANDIDATE EXPLANATION =================\n")


# Run this to see one random candidate explanation
explain_random_candidate(
    model=linear_svm,
    vectorizer=vectorizer,
    feature_names=feature_names,
    coefs=coefs,
    df=df,
    X_text=X_test_text,
    y_true=y_test,
    gender_series=test_gender,
    fairness_df=fairness_df,
    name_col=NAME_COL,
)


================= CANDIDATE EXPLANATION =================

Candidate: Joseph  (index 5940)
Applied for: Manager- Human Resource Management (HRM)

Inferred group (from name): male
True label in dataset      : REJECT
Model prediction           : REJECT
Decision score (margin)    : -0.469

Requirement satisfaction:
  • Skills requirement met?       NO
  • Experience requirement met?   NO
  • Education requirement met?    NO
  • Experience gap (cand - req):  -3.0 years
  • Education gap (cand - req):   -1.0 level(s)
  • Skill overlap (of required):  0.0%
  • Missing required skills:      1
  • Job age requirement:          "Age 25 to 40 years"

Overall requirement-based verdict:
  → This candidate looks WEAK / borderline based on requirements / matched_score.
  Matched score: 0.333

How the model decided (from text):
  The model chose to REJECT this candidate.

Top words/phrases that helped:
   + 'management' (towards shortlist, contribution 0.0569)
   + 'engineering' (towards shortlist, 